In [3]:
import os
import pandas as pd
import tensorflow
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization

In [ ]:
train = pd.read_csv(r"..\dataset\jigsaw-toxic-comment-classification-challenge\train.csv\train.csv")
test = pd.read_csv(r"..\dataset\jigsaw-toxic-comment-classification-challenge\test.csv\test.csv")
test_lable= pd.read_csv(r"..\dataset\jigsaw-toxic-comment-classification-challenge\test_labels.csv\test_labels.csv")
train.head()

In [4]:
x = train['comment_text']
y = train[train.columns[2:]].values
MAX_FEATURES = 200000  # number of words
vectorizer = TextVectorization(max_tokens = MAX_FEATURES,
                               output_mode = 'int',
                               output_sequence_length = 1800
                               )

In [9]:
vectorized_text = vectorizer(x.values)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(16)

In [11]:
train = dataset.take(int(len(dataset) * .7))
val = dataset.skip(int(len(dataset) * .7))
test = dataset.skip(int(len(dataset) * .9)).take(int(len(dataset) * .1))

In [12]:
train_generator = train.as_numpy_iterator()

# 2. Creat sequential model

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [19]:
model = Sequential([Embedding(MAX_FEATURES+1, 32),
                    Bidirectional(LSTM(32,activation='tanh')),
                    Dense(128,activation='relu'),
                    Dense(256,activation='relu'),
                    Dense(128,activation='relu'),
                    Dense(6,activation='sigmoid')])
model.compile(loss=tf.metrics.binary_crossentropy,optimizer='Adam')
model.summary()

In [21]:
history = model.fit(train, epochs=1, validation_data=val)

 957/6981 [===>..........................] - ETA: 1:15:04 - loss: 0.1038

KeyboardInterrupt: 